In [1]:
just_checking_integrity=False

In [2]:
import gc
import pandas as pd
import scipy.sparse as sps
import numpy as np

In [3]:
URM_test=sps.load_npz("../dataset/processed_data/URM_test_final.npz")

In [4]:
URM_test.shape

(4439967, 23690)

In [5]:
np.ravel(URM_test.sum(axis=1)).shape

(4439967,)

In [6]:
Session_ids=np.argwhere(np.ravel(URM_test.sum(axis=1)))

In [7]:
is_sorted = lambda a: np.all(a[:-1] <= a[1:])

In [8]:
is_sorted(Session_ids)

True

In [9]:
dataset=None
dataset_max=None
#["UCF","ICBF","Graph","ICF","UCF_W","ICBF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW"]
selected_list = ["UCF_W","ICBF_W","Graph_W","ICF_W"]
for file in selected_list:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"../dataset/candidates/traditional_recs/final/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


del df_feats
del df_max

0
loading UCF_W
16350023
loaded UCF_W
0
loading ICBF_W
14855475
loaded ICBF_W
0
loading Graph_W
8321415
loaded Graph_W
0
loading ICF_W
17771428
loaded ICF_W


In [10]:
ids=np.sort(dataset["Session_Id"].unique())
map_ids=pd.DataFrame()
map_ids["Session_Id"]=ids
if just_checking_integrity:
    map_ids["Real_Id"]=Session_ids[:len(ids)]
else:
    map_ids["Real_Id"]=Session_ids


In [11]:
dataset_max=dataset_max.merge(map_ids,on="Session_Id",how="left")
del dataset_max["Session_Id"]
dataset_max.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [12]:
dataset=dataset.merge(map_ids,on="Session_Id",how="left")
del dataset["Session_Id"]
dataset.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [14]:
#import ast
#submission_df=pd.read_csv("../dataset/candidates/NN/GRU/final/candidates.csv")
#submission_df["score"]=submission_df["score"].map(ast.literal_eval)
#submission_df["code"]=submission_df["code"].map(ast.literal_eval)
#submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
#submission_df = submission_df.explode(['code','score', 'rank'])
#submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
#submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [15]:
import ast
submission_df_tr=pd.read_csv("../dataset/candidates/NN/Transformer/final/candidates.csv")
submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [16]:
#import ast
#submission_df_LSTM=pd.read_csv("../dataset/candidates/NN/LSTM/final/candidates.csv")
#submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
#submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
#submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
#submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
#submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
#submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [17]:
#dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
#del submission_df
dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
del submission_df_tr
#dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")


In [18]:
dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")
del dataset_max

In [19]:
dataset.fillna(0,inplace=True)

/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_40463/2573695804.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset.fillna(0,inplace=True)


In [20]:
gc.collect()

1595

In [21]:
dataset_macro_features = pd.read_csv('../dataset/processed_data/macro_feats_NN_final.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')

In [22]:
# dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

In [23]:
dataset.head()

,Item_ID,Score_UCF_W,Score_ICBF_W,Score_Graph_W,Score_ICF_W,Session_Id,score_transformer,rank_transformer,Max_Score_UCF_W,Max_Score_ICBF_W,...,Max_Score_ICF_W,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,11,0.034741,0.115033,0.006554,0.311263,61,0.0,0.0,206.49619,3.357151,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,19,0.017621,0.000000,0.000000,0.000000,61,0.0,0.0,206.49619,3.357151,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,23,0.040097,0.072503,0.007389,0.325638,61,0.0,0.0,206.49619,3.357151,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,25,0.024129,0.081094,0.004554,0.006163,61,0.0,0.0,206.49619,3.357151,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,31,0.000000,0.081091,0.000000,0.000000,61,0.0,0.0,206.49619,3.357151,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [24]:
gc.collect()

0

In [25]:
dataset=dataset.sort_values("Session_Id")

In [26]:
gc.collect()

0

In [27]:
import lightgbm

In [28]:
k=10
result = []

In [30]:
for index in range(k):
    print(f"Current iteration: {str(index + 1)} of {str(k)}")
    model = lightgbm.Booster(model_file=f'../rankers/model_{index}.txt')
    result.append(model.predict(dataset.drop(["Session_Id","Item_ID"],axis=1)))

result = np.array(result)
result

Current iteration: 1 of 10
Current iteration: 2 of 10
Current iteration: 3 of 10
Current iteration: 4 of 10
Current iteration: 5 of 10
Current iteration: 6 of 10
Current iteration: 7 of 10
Current iteration: 8 of 10
Current iteration: 9 of 10
Current iteration: 10 of 10


array([[-0.74351514, -1.45082936, -1.45082936, ..., -0.80632916,
        -0.64023872, -0.80337623],
       [-1.9128246 , -3.78301296, -3.78301296, ..., -1.96244449,
        -1.8486818 , -2.17296441],
       [-1.81086673, -3.80773992, -3.80773992, ..., -1.93541287,
        -1.87554498, -2.07543634],
       ...,
       [-1.989934  , -4.04729694, -4.04729694, ..., -2.00334221,
        -1.91698813, -2.1321476 ],
       [-1.91660844, -4.04409614, -4.04409614, ..., -2.12512804,
        -1.9170351 , -2.2330382 ],
       [-1.89461755, -3.87826511, -3.87826511, ..., -2.03053895,
        -1.91085117, -2.05114005]])

In [31]:
dataset["score"]=np.mean(result ,axis=0)
# dataset["score"]=np.median(result ,axis=0)

In [32]:
dataset=dataset[["Session_Id","Item_ID","score"]]
gc.collect()

1464

In [33]:
dataset.head()

,Session_Id,Item_ID,score
0,61,11,-1.753639
389,61,3383,-3.625104
388,61,3382,-3.625104
387,61,3381,-3.625104
386,61,3380,-3.163024


In [34]:
dataset=dataset.groupby(['Session_Id']).agg(list).reset_index()

In [35]:
def get_indices(x):
    x=np.array(x)
    indices_unsorted=np.argpartition(x, -100)[-100:]
    x=x[indices_unsorted]
    indices=indices_unsorted[np.argsort(-x)]
    return indices

In [36]:
dataset["indices"]=dataset["score"].apply(lambda x: get_indices(x))

In [37]:
dataset.iloc[0]

Session_Id                                                   61
Item_ID       [11, 3383, 3382, 3381, 3380, 3379, 3378, 3377,...
score         [-1.7536389591013417, -3.625104368607208, -3.6...
indices       [315, 232, 326, 146, 366, 389, 484, 396, 44, 5...
Name: 0, dtype: object

In [38]:
dataset.iloc[0]["Item_ID"][536]

1955

In [39]:
def get_items(x):
    items=np.array(x[0])
    return items[x[1]]

In [40]:
dataset["predictions"]=dataset[["Item_ID","indices"]].apply(get_items,axis=1)

/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_40463/744460501.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  items=np.array(x[0])
/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_40463/744460501.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return items[x[1]]


In [41]:
dataset=dataset[["Session_Id","predictions"]]
dataset["rank"]=dataset.apply(lambda x:tuple([rank for rank in range(1, 101)]) ,axis=1)

In [42]:
dataset

,Session_Id,predictions,rank
0,61,"[1045, 3586, 1563, 4574, 292, 143, 2995, 699, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"[4097, 3564, 3778, 2346, 4119, 3303, 4756, 232...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"[4146, 4148, 1936, 2330, 2704, 1652, 4162, 332...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"[3338, 4202, 2769, 16, 4990, 1437, 1245, 3026,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"[3691, 3555, 2970, 2899, 3441, 3599, 2268, 465...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439646,"[270, 191, 1468, 3037, 707, 3656, 3618, 1827, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439648,"[4145, 3901, 3223, 4586, 2491, 2234, 3550, 242...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439675,"[4865, 3116, 3166, 779, 4966, 1169, 3445, 1906...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439868,"[519, 4740, 292, 2285, 1972, 3414, 3586, 750, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [43]:
dataset["predictions"]=dataset["predictions"].apply(tuple)

In [44]:
dataset=dataset[["Session_Id","predictions","rank"]]

In [45]:
dataset.columns=["session_id","code","rank"]

In [46]:
dataset.head()

,session_id,code,rank
0,61,"(1045, 3586, 1563, 4574, 292, 143, 2995, 699, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"(4097, 3564, 3778, 2346, 4119, 3303, 4756, 232...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"(4146, 4148, 1936, 2330, 2704, 1652, 4162, 332...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"(3338, 4202, 2769, 16, 4990, 1437, 1245, 3026,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"(3691, 3555, 2970, 2899, 3441, 3599, 2268, 465...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [47]:
submission_df = dataset.explode(['code', 'rank']).astype(int)

In [48]:
mapping= pd.read_csv('../dataset/processed_data/map.csv')
submission_df=submission_df.merge(mapping,on="code",how="left")
del submission_df["code"]
submission_df=submission_df[["session_id","item_id","rank"]]

In [49]:
print(len(submission_df))

5000000


In [50]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 3 columns):
 #   Column      Dtype
---  ------      -----
 0   session_id  int64
 1   item_id     int64
 2   rank        int64
dtypes: int64(3)
memory usage: 114.4 MB


In [51]:
submission_df.isna().sum()

session_id    0
item_id       0
rank          0
dtype: int64

In [52]:
submission_df.to_csv(
    f'../final_sub.csv',
    index=False
    )